In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException


driver = webdriver.Chrome()


driver.get("https://www.howstat.com/cricket/Statistics/Players/PlayerCountryList.asp")


wait = WebDriverWait(driver, 10)

country_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "cboCountry")))


specific_countries = ["Afghanistan", "Australia", "Bangladesh", "England", "India", "Ireland", "New Zealand", "Pakistan", "South Africa", "Sri Lanka", "West Indies", "Zimbabwe"]


all_players_data = []


for country in specific_countries:

    select_country = Select(country_dropdown)
    select_country.select_by_visible_text(country)

    
    driver.find_element(By.NAME, "cboCountry").click()
    
    format_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "cboFormat")))
    select_format = Select(format_dropdown)
    select_format.select_by_visible_text("T20I")

    
    table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "TableLined")))

    
    rows = table.find_elements(By.TAG_NAME, "tr")

    
    for i in range(1,len(rows)):  
        try:
            
            table = driver.find_element(By.CLASS_NAME, "TableLined")
            rows = table.find_elements(By.TAG_NAME, "tr")
            cols = rows[i].find_elements(By.TAG_NAME, "td")
            player_info = [col.text for col in cols]
            player_info.insert(2, country)  
            all_players_data.append(player_info)
        except StaleElementReferenceException:
            continue

    
    driver.get("https://www.howstat.com/cricket/Statistics/Players/PlayerCountryList.asp")
    country_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "cboCountry")))


driver.quit()


columns = ["Player Name", "Known as","Country", "Date of Birth","started career", "Matches", "Innings", "Runs","100s","Highest Score","batting average", "wickets","4W","Bowling Average", "E/R","Best"]
df = pd.DataFrame(all_players_data, columns=columns)


df.to_csv("players_data_t20i_specific_countries.csv", index=False)

print("Data scraped and saved to players_data_t20i_specific_countries.csv")


Data scraped and saved to players_data_t20i_specific_countries.csv


In [ ]:
#NextSteps:

In [1]:
import os
import pandas as pd
import numpy as np
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
df=pd.read_csv("C:\\Users\\Harshita\\Documents\\Scraping\\players_info.csv")
df

,player_name,known_as,country,date_of_birth,started_career,matches,innings,runs,hundreds,highest_score,...,wickets,four_wicket_hauls,bowling_average,economy_rate,best_bowling_figures,team_id,debuted,retired,state,ImageURL
0,Afsar Zazai*,Afsar Zazai,AFGHANISTAN,1993-08-10,2013-,8.0,8.0,141.0,0.0,48*,...,0.0,0.0,-,-,NaN,1,2013,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
1,Aftab Alam,Aftab Alam,AFGHANISTAN,1992-11-30,2012-2018,12.0,5.0,2.0,0.0,1,...,11.0,0.0,29.45,7.93,2/23,1,2012,2018,inactive,None
2,Amir Hamza*,Amir Hamza,AFGHANISTAN,1991-08-15,2013-2021,33.0,9.0,40.0,0.0,21*,...,30.0,0.0,25.07,6.77,3/39,1,2013,2021,inactive,None
3,Asghar Afghan*,Asghar Afghan,AFGHANISTAN,1987-12-22,2010-,75.0,68.0,1382.0,0.0,62*,...,1.0,0.0,4.00,6.00,1/4,1,2010,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
4,Azmatullah Omarzai*,Azmatullah Omarzai,AFGHANISTAN,2000-03-24,2022-,44.0,34.0,399.0,0.0,33*,...,28.0,1.0,31.46,8.50,4/9,1,2022,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,B V Vitori,Brian Vitori,ZIMBABWE,1990-02-22,2012-2016,11.0,2.0,8.0,0.0,7,...,4.0,0.0,80.75,8.14,1/24,12,2012,2016,inactive,None
1133,M N Waller,Malcolm Waller,ZIMBABWE,1984-09-28,2011-2018,32.0,31.0,613.0,0.0,68*,...,0.0,0.0,-,8.40,0/4,12,2011,2018,inactive,None
1134,N R Welch*,Nicholas Welch,ZIMBABWE,1998-02-05,2023-,7.0,7.0,97.0,0.0,25*,...,0.0,0.0,-,-,NaN,12,2023,0,active,None
1135,S C Williams*,Sean Williams,ZIMBABWE,1986-09-26,2006-,81.0,80.0,1691.0,0.0,66*,...,48.0,0.0,28.63,6.94,3/15,12,2006,0,active,None


In [ ]:
#Rearranging columns

In [7]:
last_col = df.columns[-1]
new_order = [last_col] + list(df.columns[:-1])
df = df[new_order]

In [9]:
df.tail()

,PlayerID,player_name,known_as,country,date_of_birth,started_career,matches,innings,runs,hundreds,...,wickets,four_wicket_hauls,bowling_average,economy_rate,best_bowling_figures,team_id,debuted,retired,state,ImageURL
1132,1133,B V Vitori,Brian Vitori,ZIMBABWE,1990-02-22,2012-2016,11.0,2.0,8.0,0.0,...,4.0,0.0,80.75,8.14,1/24,12,2012,2016,inactive,None
1133,1134,M N Waller,Malcolm Waller,ZIMBABWE,1984-09-28,2011-2018,32.0,31.0,613.0,0.0,...,0.0,0.0,-,8.40,0/4,12,2011,2018,inactive,None
1134,1135,N R Welch*,Nicholas Welch,ZIMBABWE,1998-02-05,2023-,7.0,7.0,97.0,0.0,...,0.0,0.0,-,-,NaN,12,2023,0,active,None
1135,1136,S C Williams*,Sean Williams,ZIMBABWE,1986-09-26,2006-,81.0,80.0,1691.0,0.0,...,48.0,0.0,28.63,6.94,3/15,12,2006,0,active,None
1136,1137,C Zhuwawo,Cephas Zhuwawo,ZIMBABWE,1984-12-15,2008-2018,7.0,7.0,48.0,0.0,...,1.0,0.0,1.00,3.00,1/1,12,2008,2018,inactive,None


In [11]:
df['team_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)